# Call Analytics with Amazon Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Overview

In this example, you will use Amazon Bedrock to summarize and analyze the quality of a transcript of a customer service call using an LLM. A sample transcript is provided in the './data/' folder to get started. You can also use your own transcript file. This example utilizes Amazon Bedrock, Anthropic Claude 2.0 large language model (LLM), LangChain framework, and Pydantic parser to summarize and analyze the quality of customer service call transcripts. 

### Use case

XYZ is a travel booking and vacation experience company. Customers can contact XYZ customer service representatives (CSRs) by phone to book new trips or modify existing reservations. These customer-CSR conversations are recorded, transcribed after the call ends, and analyzed by the call center management team to improve customer service processes and policies. The call transcription and analysis approach is also applicable for assessing customer interactions via email, chat, and other mediums.

### How does this work?

Transcript Ingestion
- Customer service call transcripts are uploaded as JSON objects to an S3 bucket. Plain text formats are also supported.  

Summarization Workflow
- The transcript is retrieved from the S3 source bucket and fed as a prompt to Claude, an AI assistant from Anthropic.  
- Leveraging natural language capabilities, Claude analyzes the dialogue and returns a JSON summary highlighting key discussion points and outcomes. This step uses LangChain for natural language processing and Pydantic for output structuring.

Quality Analysis Workflow  
- The full transcript is also analyzed by Claude against pre-defined quality criteria such as issue resolution, adherence to process standards, etc.  
- Assessment results are returned by Claude in a JSON format conforming to Pydantic schemas. Powered by LangChain for natural language comprehension and Pydantic for output formatting.

The two independent workflows allow simultaneous call summarization and quality analysis based on a single transcript ingestion event.

![](./images/call-analytics-example.png)


## Setup Environment

In [1]:
#Install LangChain
%pip install langchain==0.1.16 --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.chat_models import BedrockChat
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.output_parsers import PydanticOutputParser
from typing import List, Dict 
from enum import Enum
from pydantic import BaseModel, Field

import json
import boto3
from pathlib import Path

# Create Bedrock and S3 clients
boto3_session=boto3.session.Session()
bedrock_runtime = boto3_session.client("bedrock-runtime")
s3 = boto3.client('s3')

# Define LLM model 
#llm_modelId = "anthropic.claude-v2"
llm_modelId = "anthropic.claude-v2:1"

llm = BedrockChat(
    model_id=llm_modelId,
    model_kwargs={
        "max_tokens": 8000,
        "stop_sequences": ["\n\nHuman:"],
        "temperature": 0.4,
        "top_p": 1,
    },
    client=bedrock_runtime,
)

# Define S3 Location for customer service calls
s3_bucket = 'your-bucket-name' 
if s3_bucket == 'your-bucket-name': 
    print ("WARNING: PLease update 's3_bucket' with a name of your S3 bucket.")
s3_key_transcripts = 'call-analytics/call-transcript/'
s3_key_summary = 'call-analytics/call-summary/'
s3_key_score = 'call-analytics/call-score/'

# Define call transcript file name
call_transcript_file = 'Call Transcript Sample 1.json'


In [3]:
# Read call transcript file either from the repository or from an S3 bucket

##########################################################################
# OPTION 1: read from ./data/ folder in the repository (default)
##########################################################################
transcripts = Path("data")
transcript = (transcripts / call_transcript_file).open("r").read()

##########################################################################
# OPTION 2: read from S3 Location for customer service calls defined above
##########################################################################
#obj = s3.get_object(Bucket=s3_Bucket, Key=s3_Key_Transcripts + call_transcript_file)
#transcript = obj['Body'].read()

# Define variables to simplify code in subsequent steps 
transcript_dict = json.loads(transcript)
call_date = json.loads (transcript)['call_date']
call_time = json.loads (transcript)['call_time']

print(json.dumps(transcript_dict, indent=2))

{
  "call_ID": "12345",
  "CSR_ID": "JaneDoe123",
  "call_date": "2024-02-01",
  "call_time": "02:16:43",
  "call_transcript": [
    "CSR: Thank you for calling ABC Travel, this is Jane. How may I assist you today? ",
    "Customer: Yes, I need help with a reservation I made last week. This is unacceptable service! ",
    "CSR: I apologize for the trouble. May I have your name and reservation number to look up your booking? ",
    "Customer: It's John Smith. My reservation number is 012345. I booked a trip to Hawaii last week and just got an email that my flight was canceled! This is ridiculous. ",
    "CSR: Let me take a look at your reservation here Mr. Smith. I see that your flight from Chicago to Honolulu on March 15th was indeed canceled by the airline. I do apologize for this inconvenience. ",
    "Customer: This is unbelievable! I booked this trip months ago. How could you just cancel my flight like that? I took time off work and made so many plans. This is completely unacceptab

## Call Summarization

In this step, we utilize the LangChain framework and Pydantic parser to generate LLM output in JSON format.

Refer to [Pydantic parser](https://python.langchain.com/docs/modules/model_io/output_parsers/types/pydantic) on LangChain site for addtional details and examples. 


In [4]:
# Define a data schema for the LLM output with required attributes
class CallSummary(BaseModel):
    call_summary: str = Field(description="Call transcript summary: ")
    key_takeaways: List[str] = Field(description="Call transcript key takeaways: ")
    follow_up_actions: List[str] = Field(description="Call Transcript key action items: ")

# Define Pydantic parser based on the data schema 
summarization_parser = PydanticOutputParser(pydantic_object=CallSummary)

# Define a template for the LLM prompt with {format_instructions} and  {transcript} placeholder inputs
summarization_template = """

Please provide a summary of the following call transcript provided between <transcript></transcript> tags. 
Capture key takeaways and specific follow up actions. 

Format your response per the instructions below: 
{format_instructions} 

<transcript>{transcript}</transcript>
"""

# Incorporate the format instructions into the LLM prompt based on the prompt template
summarization_prompt = ChatPromptTemplate.from_template(summarization_template, partial_variables={"format_instructions": summarization_parser.get_format_instructions()})

In [5]:
# Review the content of the prompt with format instructions (notice the JSON example generated by Pydantic parser)
print (summarization_prompt)

input_variables=['transcript'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['transcript'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"call_summary": {"title": "Call Summary", "description": "Call transcript summary: ", "type": "string"}, "key_takeaways": {"title": "Key Takeaways", "description": "Call transcript key takeaways: ", "type": "array", "items": {"type": "string"}}, "follow_up_actions": {"title": "Follow Up Actions", "description": "Call Transcript key action items: ", "type": "a

In [6]:
# Define a function to convert the call transcript text from 'list' to 'string' format
def process_transcript(transcript:str) -> str:
    json_transcript = json.loads(transcript)
    call_transcript = "\n".join(json_transcript.get("call_transcript", []))

    return call_transcript

# Construct the chain by essembing all required components using LangChain '|' operator
summarization_chain = {"transcript": RunnableLambda(process_transcript)} | summarization_prompt | llm | summarization_parser


In [7]:
print (transcript)

{"call_ID": "12345", "CSR_ID": "JaneDoe123", "call_date": "2024-02-01", "call_time": "02:16:43", "call_transcript": ["CSR: Thank you for calling ABC Travel, this is Jane. How may I assist you today? ", "Customer: Yes, I need help with a reservation I made last week. This is unacceptable service! ", "CSR: I apologize for the trouble. May I have your name and reservation number to look up your booking? ", "Customer: It's John Smith. My reservation number is 012345. I booked a trip to Hawaii last week and just got an email that my flight was canceled! This is ridiculous. ", "CSR: Let me take a look at your reservation here Mr. Smith. I see that your flight from Chicago to Honolulu on March 15th was indeed canceled by the airline. I do apologize for this inconvenience. ", "Customer: This is unbelievable! I booked this trip months ago. How could you just cancel my flight like that? I took time off work and made so many plans. This is completely unacceptable! ", "CSR: You're absolutely right

In [8]:
# Invoke the chain; the output will contain the LLM output in JSON format
summary = summarization_chain.invoke(transcript)

print (summary.json())

{"call_summary": "The customer booked a trip to Hawaii that got canceled. The CSR apologized and processed full refunds. The customer was very frustrated and asked for a supervisor. The supervisor also apologized, took responsibility, and offered future credits. The customer said the experience was terrible and the company needs better training.", "key_takeaways": ["Customer's flight to Hawaii was canceled by the airline", "Customer was offered rebooking but requested full refunds", "Customer expressed frustration at multiple points during call", "Supervisor took responsibility and offered credits for future travel"], "follow_up_actions": ["Process full refund for customer's canceled Hawaii trip", "Assess internal procedures around managing cancellations", "Implement better training for staff on handling cancellations", "Follow up with customer regarding future travel credits"]}


In [9]:
# Extract the required values for preview: Call Summary, Key Takeaways, Follow Up Actions.
call_summary = summary.call_summary
key_takeaways = "-" + "\n-".join(summary.key_takeaways)
follow_up_actions = "-" + "\n-".join(summary.follow_up_actions)

print(f"Call Summary:\n{call_summary}\n\nKey Takeaways:\n{key_takeaways}\n\nFollow Up Actions\n{follow_up_actions}")

Call Summary:
The customer booked a trip to Hawaii that got canceled. The CSR apologized and processed full refunds. The customer was very frustrated and asked for a supervisor. The supervisor also apologized, took responsibility, and offered future credits. The customer said the experience was terrible and the company needs better training.

Key Takeaways:
-Customer's flight to Hawaii was canceled by the airline
-Customer was offered rebooking but requested full refunds
-Customer expressed frustration at multiple points during call
-Supervisor took responsibility and offered credits for future travel

Follow Up Actions
-Process full refund for customer's canceled Hawaii trip
-Assess internal procedures around managing cancellations
-Implement better training for staff on handling cancellations
-Follow up with customer regarding future travel credits


In [10]:
# Construct call summary as JSON object with all relevant attributes to be stored in S3
bedrock_response = json.loads(summary.json())
bedrock_response ["call_ID"] = transcript_dict['call_ID']
bedrock_response ["CSR_ID"] = transcript_dict['CSR_ID']
bedrock_response ["call_date"] = call_date
bedrock_response ["call_time"] = call_time
bedrock_response ["llm_model"] = llm_modelId
bedrock_response = json.dumps (bedrock_response)
print (bedrock_response)

# Write Bedrock output text to S3 object  
s3_key = s3_key_summary + "Call Summary " + call_date + " " + call_time + ".json"
s3.put_object(Body=bedrock_response, Bucket=s3_bucket, Key=s3_key )

print("Transcript summary written to S3:" + s3_key)

{"call_summary": "The customer booked a trip to Hawaii that got canceled. The CSR apologized and processed full refunds. The customer was very frustrated and asked for a supervisor. The supervisor also apologized, took responsibility, and offered future credits. The customer said the experience was terrible and the company needs better training.", "key_takeaways": ["Customer's flight to Hawaii was canceled by the airline", "Customer was offered rebooking but requested full refunds", "Customer expressed frustration at multiple points during call", "Supervisor took responsibility and offered credits for future travel"], "follow_up_actions": ["Process full refund for customer's canceled Hawaii trip", "Assess internal procedures around managing cancellations", "Implement better training for staff on handling cancellations", "Follow up with customer regarding future travel credits"], "call_ID": "12345", "CSR_ID": "JaneDoe123", "call_date": "2024-02-01", "call_time": "02:16:43", "llm_model":

ClientError: An error occurred (AllAccessDisabled) when calling the PutObject operation: All access to this object has been disabled

## Call Quality Assessment
In this step, we utilize the LangChain framework and Pydantic parser to generate LLM output in JSON format. However, we expand the formatting instructions and prompt complexity in comparison to the Call Summarization step to achieve more tailored results. Specificially:

1. The prompt template provides a list of call quality assessment categories, each with a descriptive explanation of what should be evaluated within that category. 

2. The data model for the LLM output has two levels of nesting to capture detailed scoring for each category. 

Refer to [Pydantic parser](https://python.langchain.com/docs/modules/model_io/output_parsers/types/pydantic) on LangChain site for addtional details and examples. 

In [11]:
assessment_template =  """
Evaluate call transcript against categories shown between <categories></categories> tags and provide score as 'High', 'Medium', 'Low' for each category.

Format your response per the instructions below: 
{format_instructions} 


<categories>
1. Communication Skills:
 - Clarity: How clearly and concisely does the CSR communicate information?
 - Active Listening: Does the CSR actively listen to the customer's concerns and questions?
 - Empathy: How well does the CSR demonstrate empathy and understanding towards the customer?

2. Problem Resolution:
 - Effectiveness: How well did the CSR resolve the customer's issue or answer their question?
 - Timeliness: Was the issue resolved in a reasonable amount of time?

3. Product Knowledge:
 - Familiarity: Does the CSR have a good understanding of the company's products and services?
 - Accuracy: How accurate and precise are the answers provided by the CSR?

4. Professionalism:
 - Tone and Manner: How professional is the tone and manner of the CSR throughout the call?
 - Courtesy: Does the CSR maintain a courteous and respectful attitude towards the customer?

5. Problem Escalation:
 - Recognition: Did the CSR recognize when an issue required escalation to a higher level of support?
 - Handoff: How smoothly and effectively did the CSR transfer the call if escalation was necessary?

6. Resolution Follow-Up:
 - Follow-Up: Did the CSR provide information about any follow-up actions that would be taken?
 - Customer Satisfaction: Did the CSR inquire about the customer's satisfaction with the resolution?

7. Efficiency:
 - Call Handling Time: Was the call resolved efficiently without unnecessary delays?
 - Multi-Tasking: If applicable, did the CSR effectively handle multiple tasks during the call?

8. Adherence to Policies and Procedures:
 - Compliance: Did the CSR follow company policies and procedures in addressing the customer's issue?
 - Accuracy in Information: How well did the CSR adhere to the correct processes?

9. Technical Competence:
 - System Use: Did the CSR effectively navigate and use the customer service tools and systems?
 - Troubleshooting: How adept is the CSR at troubleshooting technical issues?

10. Customer Satisfaction:
 - Overall Satisfaction: How satisfied is the customer with the service received during the call?
 - Feedback: Did the CSR encourage the customer to provide feedback on the service?

11. Language Proficiency:
 - Clarity of Language: Was the language used by the CSR easily understandable?
 - Language Appropriateness: Did the CSR use appropriate language for effective communication?

12. Conflict Resolution:
 - Handling Difficult Customers: How well did the CSR manage and resolve conflicts with upset or frustrated customers?
 - De-escalation Skills: Did the CSR employ de-escalation techniques when needed?
<categories>

Here is the call transcript:
<transcript>{transcript}</transcript>

"""

In [12]:
# Define a data schema for the LLM output with required attributes

class ScoreValue(Enum):
    High = "High"
    Medium = "Medium"
    Low = "Low"

class Score(BaseModel):
    score: ScoreValue
    score_explanation: str

class Evaluation(BaseModel):
    Communication_Skills: Score
    Problem_Resolution: Score
    Product_Knowledge: Score
    Professionalism: Score
    Problem_Escalation: Score
    Resolution_Follow_Up: Score
    Efficiency: Score
    Adherence_to_Policies_and_Procedures: Score
    Technical_Competence: Score
    Customer_Satisfaction: Score
    Language_Proficiency: Score
    Conflict_Resolution: Score
    
# Define Pydantic parser based on data schema 
assessment_parser = PydanticOutputParser(pydantic_object=Evaluation)

In [13]:
# Incorporate the format instructions into the LLM prompt based on the prompt template
assessment_prompt = ChatPromptTemplate.from_template(assessment_template, partial_variables={"format_instructions": assessment_parser.get_format_instructions()})

# Construct the chain by essembing all required components via LangChain '|' operator
assessment_chain = {"transcript": RunnableLambda(process_transcript)} | assessment_prompt | llm | assessment_parser

In [14]:
# Invoke the chain; the output will contain the LLM output in JSON format
call_assessment = assessment_chain.invoke(transcript)

In [15]:
# Preview score values for each category provided in the LLM output
for category, score in call_assessment:
    print(f"{category}: score={score.score.value}, explanation={score.score_explanation}\n")

Communication_Skills: score=High, explanation=The CSR and supervisor communicated clearly, listened actively, and showed empathy. They apologized sincerely and took responsibility.

Problem_Resolution: score=High, explanation=The CSR quickly resolved the issue by providing full refunds. The supervisor also offered credits for future travel.

Product_Knowledge: score=High, explanation=The CSR was familiar with the policies around flight cancellations and refunds. The responses were accurate.

Professionalism: score=High, explanation=Both the CSR and supervisor maintained a professional, courteous tone even when the customer was frustrated.

Problem_Escalation: score=High, explanation=The CSR appropriately recognized the need to involve a supervisor. The handoff was smooth.

Resolution_Follow_Up: score=High, explanation=The CSR provided clear follow-up on the refund status. The supervisor followed up on customer satisfaction.

Efficiency: score=High, explanation=The issue was resolved qu

In [16]:
# Preview content of the call_assessment JSON object
print (type(json.loads(call_assessment.json())))
print (call_assessment.json())

<class 'dict'>
{"Communication_Skills": {"score": "High", "score_explanation": "The CSR and supervisor communicated clearly, listened actively, and showed empathy. They apologized sincerely and took responsibility."}, "Problem_Resolution": {"score": "High", "score_explanation": "The CSR quickly resolved the issue by providing full refunds. The supervisor also offered credits for future travel."}, "Product_Knowledge": {"score": "High", "score_explanation": "The CSR was familiar with the policies around flight cancellations and refunds. The responses were accurate."}, "Professionalism": {"score": "High", "score_explanation": "Both the CSR and supervisor maintained a professional, courteous tone even when the customer was frustrated."}, "Problem_Escalation": {"score": "High", "score_explanation": "The CSR appropriately recognized the need to involve a supervisor. The handoff was smooth."}, "Resolution_Follow_Up": {"score": "High", "score_explanation": "The CSR provided clear follow-up on 

In [17]:
# Construct the call summary as JSON object with all relevant attributes and save it to S3
bedrock_response = json.loads(call_assessment.json())
bedrock_response ["call_ID"] = transcript_dict['call_ID']
bedrock_response ["CSR_ID"] = transcript_dict['CSR_ID']
bedrock_response ["call_date"] = call_date
bedrock_response ["call_time"] = call_time
bedrock_response ["llm_model"] = llm_modelId
bedrock_response = json.dumps (bedrock_response)
print (bedrock_response)

# Write Bedrock output text to S3 object  
s3_key = s3_key_score + "Call score " + call_date + " " + call_time + ".json"
s3.put_object(Body=bedrock_response, Bucket=s3_bucket, Key=s3_key )

print("Transcript score assessment written to S3:" + s3_key)

{"Communication_Skills": {"score": "High", "score_explanation": "The CSR and supervisor communicated clearly, listened actively, and showed empathy. They apologized sincerely and took responsibility."}, "Problem_Resolution": {"score": "High", "score_explanation": "The CSR quickly resolved the issue by providing full refunds. The supervisor also offered credits for future travel."}, "Product_Knowledge": {"score": "High", "score_explanation": "The CSR was familiar with the policies around flight cancellations and refunds. The responses were accurate."}, "Professionalism": {"score": "High", "score_explanation": "Both the CSR and supervisor maintained a professional, courteous tone even when the customer was frustrated."}, "Problem_Escalation": {"score": "High", "score_explanation": "The CSR appropriately recognized the need to involve a supervisor. The handoff was smooth."}, "Resolution_Follow_Up": {"score": "High", "score_explanation": "The CSR provided clear follow-up on the refund stat

ClientError: An error occurred (AllAccessDisabled) when calling the PutObject operation: All access to this object has been disabled

## The End of the Notebook